In [1]:
'''
CHATBOT - final NAI project
All needed info avalible in read.me or in presentation
Programming team : Oktawian Filipkowski
'''

'''
Required installs
'''
!pip install wikipedia
!pip install translate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
'''
Required imports
'''
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from translate import Translator
import random
import wikipedia
import nltk
import json
import pickle
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
'''
NLTK downloads
'''
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
'''
Creating fucntions needed in wikipedia part of the bot
'''
lemmatizer = WordNetLemmatizer()

def lemma(sent):
    sentence_tokens = nltk.word_tokenize(sent.lower())
    pos_tags = nltk.pos_tag(sentence_tokens)

    sentence_lemmas = []
    for token, pos_tag in zip(sentence_tokens, pos_tags):
        if pos_tag[1][0].lower() in ['n', 'v', 'a', 'r']:
            lemma = lemmatizer.lemmatize(token, pos_tag[1][0].lower())
            sentence_lemmas.append(lemma)

    return sentence_lemmas

def process(text, question):
  sentence_tokens = nltk.sent_tokenize(text)
  sentence_tokens.append(question)

  tv = TfidfVectorizer(tokenizer=lemma)
  tf = tv.fit_transform(sentence_tokens)
  values = cosine_similarity(tf[-1], tf)
  index = values.argsort()[0][-2]
  values_flat = values.flatten()
  values_flat.sort()
  coeff = values_flat[-2]
  if coeff > 0.3:
    return sentence_tokens[index]

In [4]:
'''
Creating functions needed to translate
'''
def translate(translate):
  translator= Translator(from_lang="english",to_lang="polish")
  MyString = translate
  i = len(MyString)
  if i%2 == 0:
      str1 = MyString[0:i//2]
      str2 = MyString[i//2:]
      sentence = str1
      sentence2 = str2
      translation = translator.translate(sentence)
      translation2 = translator.translate(sentence2)
      print(translation, translation2)
  else:
      str1 = MyString[0:(i//2+1)]
      str2 = MyString[(i//2+1):]
      sentence = str1
      sentence2 = str2
      translation = translator.translate(sentence)
      translation2 = translator.translate(sentence2)
      print(translation, translation2)

def translate_eng(translate):
  translator= Translator(from_lang="polish",to_lang="english")
  MyString = translate
  i = len(MyString)
  if i%2 == 0:
      str1 = MyString[0:i//2]
      str2 = MyString[i//2:]
      sentence = str1
      sentence2 = str2
      translation = translator.translate(sentence)
      translation2 = translator.translate(sentence2)
      translated = (translation +' '+ translation2)
      return translated
  else:
      str1 = MyString[0:(i//2+1)]
      str2 = MyString[(i//2+1):]
      sentence = str1
      sentence2 = str2
      translation = translator.translate(sentence)
      translation2 = translator.translate(sentence2)
      translated = (translation +' '+ translation2)
      return translated

In [5]:
'''
Creating objects needed in learning part of the bot
'''
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [6]:
'''
Tokenization, adding docs to corpus, lemmatization.
Classes will be counted and unique lemmatized words will be printed for reference
'''
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

105 documents
43 classes ['Location', 'Weather', 'animals', 'anime', 'anime_reccomendation', 'birthday', 'cats', 'commission', 'configuration', 'connect_people', 'customer_satisfaction', 'dogs', 'email', 'factors_impacting_sale', 'feelings', 'gadgets', 'game_info', 'games', 'games_reccomendation', 'goodbye', 'greeting', 'highest_grossing', 'hours', 'invalid', 'key_customers', 'leave', 'maintainence', 'manga', 'music', 'name', 'noans', 'options', 'predict_delay', 'predict_performance', 'project_handling_queries', 'rock', 'search_department', 'solve_problems', 'thanks', 'turnover', 'vegetables', 'version_update', 'vitamins']
184 unique lemmatized words ["'s", ',', 'a', 'about', 'adopt', 'aggresive', 'am', 'an', 'animal', 'anime', 'any', 'anyone', 'are', 'at', 'avalible', 'awesome', 'be', 'been', 'best', 'bored', 'born', 'break', 'bye', 'cafeteria', 'call', 'can', 'canteen', 'chat', 'chatting', 'commission', 'company', 'complaint', 'computer', 'configuration', 'configure', 'conflict', 'co

In [7]:
'''
Creation of training data and empty array
'''
training = []

output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-7-43399ea6a252>:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [8]:
'''
Creating training model and saving it
'''
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
21/21 [==============================] - 1s 2ms/step - loss: 3.7813 - accuracy: 0.0095 
Epoch 2/200
21/21 [==============================] - 0s 2ms/step - loss: 3.7190 - accuracy: 0.0381
Epoch 3/200
21/21 [==============================] - 0s 2ms/step - loss: 3.6601 - accuracy: 0.0952
Epoch 4/200
21/21 [==============================] - 0s 2ms/step - loss: 3.5939 - accuracy: 0.1143
Epoch 5/200
21/21 [==============================] - 0s 2ms/step - loss: 3.4880 - accuracy: 0.1048
Epoch 6/200
21/21 [==============================] - 0s 3ms/step - loss: 3.4155 - accuracy: 0.1238
Epoch 7/200
21/21 [==============================] - 0s 6ms/step - loss: 3.3691 - accuracy: 0.1238
Epoch 8/200
21/21 [==============================] - 0s 5ms/step - loss: 3.2175 - accuracy: 0.1333
Epoch 9/200
21/21 [==============================] - 0s 4ms/step - loss: 3.1253 - accuracy: 0.1619
Epoch 10/200
21/21 [==============================] - 0s 6ms/step - loss: 3.0230 - accuracy: 0.2571
Epoch 11

In [9]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
'''
Filtering and sorting predictions by probability
'''
def predict_class(sentence, model):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [10]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [18]:
while True:
    langg = None
    while langg == None:
      langg = input('for english enter "eng", for polish enter "pol"').lower()
      if langg == 'pol':
        print('wybrano pol')
        wikipedia.set_lang("pl")
        break
      elif langg == 'eng':
        print('eng selected')
        wikipedia.set_lang("en")
        break
      else:
        print("error")
        langg = None

    if langg == 'eng':
      print('Im simple bot, and can only talk about selected topics :(\n'
            'What do you want to talk about? '
            'chose and type from :\n'
            'animals, manga, vegetables, music, games \n'
            'type "quit" to end conversation')
      category = input('chose: ').lower()
    else:
      print('Jestem prostym botem i mam ograniczony zasob tematow :(\n'
            'Wybierz sposrod ponizszych tematow\n'
            'zwierzeta, manga, warzywa, muzyka, gry\n'
            'wpisz "quit" by zakonczyc rozmowe\n')
      category = input('wybor: ').lower()

    if category == 'animals' or category == 'zwierzeta' or category == 'manga' or category == 'vegetables' or category == 'warzywa' or category == 'music' or category == 'muzyka' or category == 'games' or category == 'gry':
            while True:
                if langg == 'eng':
                  print(f'{category} chosen, if you want to change category and language, type "category"')
                  talking = input(f'What {category} interests you? ').lower()
                else:
                  print(f'wybrano {category},by zmienic wpisz "category"')
                  talking = input(f'Jakie {category} interesuje ciebie ').lower()
                if talking == 'category':
                    break
                else:
                    while True:
                      if langg == 'eng':
                        question = input(f'You want to ask about {talking}? (If not type anything to change it)').lower()
                      else:
                        question = input(f'Chcesz zapytac o {talking}? (jesli nie to wpisz cokolwiek by wrocic do menu wyzej)').lower()
                      if question:
                          break
                      else:
                          page = (category + ' ' + talking)
                          try:
                              page = wikipedia.page(page)
                          except wikipedia.DisambiguationError as e:
                              error_page = random.choice(e.options)
                              page = random.choice(error_page)
                          except wikipedia.PageError:
                              error_page = wikipedia.search(talking)
                              save = random.choice(error_page)
                              page = wikipedia.summary(save)
                              text = wikipedia.page(save).content
                          try:
                            text = wikipedia.page(page).content
                          except wikipedia.WikipediaException:
                            page = wikipedia.page(talking)

                          while True:
                              if langg == 'eng':
                                question2 = input('Ask away!, if you want to change topic, type "topic"\n')
                              else:
                                question2 = input('Pytaj!, by zmienic temat, wpisz "topic"\n')
                              output = process(text, question2)
                              if question2 == 'topic':
                                  break
                              elif output:
                                  print(output)
                              else:
                                  if langg == 'eng':
                                    message = question2
                                    ints = predict_class(message, model)
                                    res = getResponse(ints, intents)
                                    try:
                                          ints = predict_class(message, model)
                                          res = getResponse(ints, intents)
                                          print(res)
                                    except IndexError:
                                          print("What?")
                                    except RuntimeError:
                                          print("What??")
                                  else:
                                    try:
                                        message = translate_eng(question2)
                                        ints = predict_class(message, model)
                                        res = getResponse(ints, intents)
                                        ints = predict_class(message, model)
                                        res = getResponse(ints, intents)
                                        print(translate(res))
                                    except IndexError:
                                          print("Co?")
                                    except RuntimeError:
                                          print("Co??")



    elif category == 'quit':
        print("bye!")
        break
    else:
        print("wrong choice!")

for english enter "eng", for polish enter "pol"pol
wybrano pol
Jestem prostym botem i mam ograniczony zasob tematow :(
Wybierz sposrod ponizszych tematow
zwierzeta, manga, warzywa, muzyka, gry
wpisz "quit" by zakonczyc rozmowe

wybor: zwierzeta
wybrano zwierzeta,by zmienic wpisz "category"
Jakie zwierzeta interesuje ciebie psy
Chcesz zapytac o psy? (jesli nie to wpisz cokolwiek by wrocic do menu wyzej)
page
Pytaj!, by zmienic temat, wpisz "topic"
co to psy
Wykorzystywane są jako:

psy myśliwskie
tropowce (bloodhound)
legawce (pudelpointer)
płochacze (płochacz niemiecki)
posokowce (posokowiec bawarski)
gończe (gończy Hamiltona)
norowce (jamnik)
psy aportujące (curly coated retriever)
psy ratownicze (nowofundland)
psy pasterskie (anatolian)
psy zaganiające (bouvier des Flandres)
psy stróżujące (mastif tybetański)
psy stróżująco-obronne (doberman)
psy służbowe wykorzystywane na potrzeby wojska (owczarek belgijski, czarny terier rosyjski), policji (owczarek niemiecki), służb celnych (pies 